In [1]:
from datasets import load_dataset

huggingface_dataset_name = "multi_news"
dataset = load_dataset(huggingface_dataset_name)
dataset

/Users/soli/Documents/Projects/llm/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 5622/5622 [00:00<00:00, 18314.74 examples/s]


DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [2]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [5]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [6]:
dataset["train"]

Dataset({
    features: ['document', 'summary'],
    num_rows: 44972
})

In [10]:
dataset["train"][0]["document"]

'National Archives \n \n Yes, it’s that time again, folks. It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. \n \n A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. \n \n Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you’re here, why don’t you sign up to follow us on Twitter. \n \n Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be gr

In [11]:
dataset["test"]

Dataset({
    features: ['document', 'summary'],
    num_rows: 5622
})

In [12]:
dataset["validation"]

Dataset({
    features: ['document', 'summary'],
    num_rows: 5622
})

In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

model_name="google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer

T5TokenizerFast(name_or_path='google/flan-t5-base', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>'

In [59]:
example_indices = [7]
dash_line = "--------------------------------"

for i, index in enumerate(example_indices):
    document = dataset["test"][index]["document"]
    summary = dataset["test"][index]["summary"]

    prompt = f"""
{document}

Summary of the text above using only one short sentence:

    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors="pt")
    generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            generation_config=generation_config,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print("Document ", index)
    print(dash_line)
    print(f"HUMAN SUMMARY:\n{summary}")
    print(dash_line)    
    print(f"MODEL SUMMARY:\n{output}\n")

--------------------------------
Document  7
--------------------------------
HUMAN SUMMARY:
– A Picasso painting that was found to have vanished from a Paris museum more than a decade ago has turned up—in the US, in a package shipped from Belgium. Its papers identified it as a $37 "art craft/toy" and also included the line "Joyeux Noel," or Merry Christmas, the New York Times reports. A man named "Robert" attempted to send the package to a climate-controlled warehouse in Queens, New York, in December, but custom officials at the Port of Newark seized what turned out to be Picasso's 1911 La Coiffeuse (The Hairdresser). French museum officials traveled to New York last month and confirmed the find is indeed the missing Picasso work, which the Centre Georges Pompidou realized was missing from its storerooms in 2001 following a loan request; it was then valued at more than $2.5 million. Court documents don't specify whether the sender or would-be recipient have been identified, NBC News r